In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from malaya_speech.torch_model.vits.model_infer import SynthesizerTrn
from malaya_boilerplate.train.config import HParams
import json

2023-08-25 16:35:49.812271: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 16:35:49.929133: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-25 16:35:50.673335: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-25 16:35:50.673370: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [3]:
config = 'malay_vits/ms-husein.json'

In [4]:
with open(config) as fopen:
    hps = HParams(**json.load(fopen))

In [5]:
from malaya_speech.utils.text import TTS_SYMBOLS
import torch

In [6]:
model = SynthesizerTrn(len(TTS_SYMBOLS),
                                   hps.data.filter_length // 2 + 1,
                                   hps.train.segment_size // hps.data.hop_length,
                                   **hps.model)

In [22]:
!ls malay_vits/logs/husein/G_*.pth

malay_vits/logs/husein/G_130000.pth  malay_vits/logs/husein/G_140000.pth
malay_vits/logs/husein/G_135000.pth


In [23]:
pth = 'malay_vits/logs/husein/G_140000.pth'

In [24]:
model.load_state_dict(torch.load(pth, map_location=torch.device('cpu'))['model'])

<All keys matched successfully>

In [25]:
_ = model.eval()

In [11]:
from malaya_speech.torch_model.vits.commons import intersperse
import malaya_speech

In [12]:
normalizer = malaya_speech.utils.text.TextIDS(pad_to = None, is_lower = False)

In [13]:
text = 'Haqkiem adalah pelajar tahun akhir yang mengambil Ijazah Sarjana Muda Sains Komputer Kecerdasan Buatan.'

In [26]:
text2 = 'KUALA TERENGGANU - Kerajaan didakwa ketandusan isu sehingga mengaitkan pemimpin Perikatan Nasional (PN) dengan kenyataan berunsur perkauman.'

In [42]:
t, ids = normalizer.normalize(text2.lower())
t

'kuala terengganu - kerajaan didakwa ketandusan isu sehingga mengaitkan pemimpin perikatan nasional , pn , dengan kenyataan berunsur perkauman .'

In [43]:
ids = intersperse(ids, 0)

In [44]:
ids = torch.LongTensor(ids)
ids_lengths = torch.LongTensor([ids.size(0)])
ids = ids.unsqueeze(0)

In [45]:
audio = model.infer(
    ids,
    ids_lengths,
    noise_scale=0.1,
    noise_scale_w=0.1,
    length_scale=1.0,
)

In [46]:
audio = audio[0][0,0].detach().numpy()

In [47]:
import IPython.display as ipd
ipd.Audio(audio, rate = 22050)

In [48]:
torch.save(model.state_dict(), 'husein.pth')

In [49]:
from malaya_boilerplate.huggingface import upload_dict

In [50]:
files_mapping = {'husein.pth': 'model.pth',
                config: 'config.json'}
upload_dict(model = 'VITS-husein', files_mapping = files_mapping, username = 'mesolitica')

409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-64e8fc3e-4225534a3bdd1a743b68465d;2f7ebeb5-3f82-4c1d-ad2c-80ed0d5f3731)

You already created this model repo


husein.pth:   0%|          | 0.00/145M [00:00<?, ?B/s]